In [4]:
from janome.tokenizer import Tokenizer
from collections import Counter
from collections import defaultdict

def get_three_words_list(sentence):  # 関数にする
    """文章を3単語の組にして返す"""
    t = Tokenizer()
    words = t.tokenize(sentence, wakati=True)
    words = [BEGIN] + words + [END]
    three_words_list = []
    for i in range(len(words) - 2):
        three_words_list.append(tuple(words[i:i+3]))
    return three_words_list

def generate_markov_dict(three_words_count):
    """マルコフ連鎖での文章生成用の辞書データを生成する"""
    markov_dict = {}
    for three_words, count in three_words_count.items():
        two_words = three_words[:2]  # 「前半2つの単語」と「次の単語」に分割
        next_word = three_words[2]
        if two_words not in markov_dict: # 辞書に存在しない場合は空データを生成
            markov_dict[two_words] = {'words': [], 'weights': []}
        markov_dict[two_words]['words'].append(next_word)  # 次の単語と回数を追加
        markov_dict[two_words]['weights'].append(count)
    return markov_dict


def get_first_words_weights(three_words_count):
    """最初の単語を選択するための辞書データを作成する"""
    first_word_count = defaultdict(int)

    for three_words, count in three_words_count.items():
        if three_words[0] == BEGIN:
            next_word = three_words[1]
            first_word_count[next_word] += count

    words = []  # 単語と重み(出現回数)を格納するリスト
    weights = []
    for word, count in first_word_count.items():
        words.append(word)  # 単語と重みをリストに追加
        weights.append(count)
            
    return words, weights

BEGIN = '__BEGIN__'  # 文の開始マーク
END = '__END__'  # 文の終了マーク

data =open('Japanese.txt', encoding="utf-8_sig") 
data1=data.read()
data.close()

sentences = data1.split('。')  # 。で文章を分割
print('文の数:', len(sentences))
sentences[:10]

import time
from tqdm import tqdm
for i in tqdm(range(50)):  # tqdm()で囲む
    time.sleep(0.5)  # 0.5秒スリープ

from tqdm import tqdm

three_words_list = []
for sentence in tqdm(sentences):  # tqdmで進捗バーを表示する
    three_words_list += get_three_words_list(sentence)
three_words_count = Counter(three_words_list)
len(three_words_count)  # 3単語の組の種類を確認

markov_dict = generate_markov_dict(three_words_count)  # 文章生成用の辞書データを作成
first_words, first_weights = get_first_words_weights(three_words_count)  # 最初の単語と出現数を取得


import random

def generate_text(fwords, fweights, markov_dict):
    """入力された辞書データを元に文章を生成する"""
    first_word = random.choices(fwords, weights=fweights)[0]  # 最初の単語を取得
    generate_words = [BEGIN, first_word]  # 文章生成用に単語を格納するリスト
    while True:
        pair = tuple(generate_words[-2:])  # 最後の2つの単語を取得
        words = markov_dict[pair]['words']  # 次の単語と重みのリストを取得
        weights = markov_dict[pair]['weights']
        next_word = random.choices(words, weights=weights)[0]  # 次の単語を取得
        if next_word == END:  # 文章が終了した場合はループを抜ける
            break
        generate_words.append(next_word)

    return ''.join(generate_words[1:])  # 単語から文章を作成

文の数: 49523


100%|████████████████████████████████████████████████████████████████████████████| 49523/49523 [55:52<00:00,  8.54it/s]


In [324]:
for _ in range(5):
    sentence = generate_text(first_words, first_weights, markov_dict)
    print(sentence)

彼女は彼には何でも読みなさい
物価が安くつく
会社へ電話できますか
私たちを行かせた
私は彼を責めることは知らない
